In [5]:
from flask import Flask, request, jsonify
import base64
import tempfile
import os
from bob.io.audio import reader
from utils import configuration, silence_detection

In [6]:
config = configuration()

In [ ]:
class iVector_model:
    def __init__(self,mfcc_feat):
        self.fubm = open(config['model_files']['full_GMM-UBM']).read()
        self.ivector_extractor = open(config['model_files']['iVector']).read()
        
        self.iVector = bob.kaldi.ivector_extract(mfcc_feat, 
                                            fubm,
                                            ivector_extractor,
                                            num_gselect=config['iVector_extract_params']['num_gselect'],
                                            min_post=config['iVector_extract_params']['min_post'],
                                            posterior_scale=config['iVector_extract_params']['posterior_scale'])

    def enroll(self,enroll_features):
        '''
        Args - self, enroll_features
                     enroll_features - MFCC features extracted with expaneded dimensions via np.expand_dims(,axis=0)
        Return - speaker model
        '''
        return self.iVector.enroll(enroll_features)

    def score(self,model,probe):
        '''
        Args - self, model , probe
                     model - Model of the speaker from the enroll function (from the already enrolled speaker)
                     probe - Projection of the feature array from the speech utterance for verification

        Return - Cosine Similarity Score beween the model and probe
        '''
        return self.iVector.score(model,probe)

In [6]:
def decode_base64(base64string):
    b64_byte = base64.b64decode(base64string)
    wav_temp = tempfile.TemporaryFile()
    
    with open("./" + str(wav_temp.name) ,'wb') as wav: 
        wav.write(b64_byte)
    
    data = bob.io.audio.reader(str(wav_temp.name))
    
    if data.rate!=16000:
        return {'status':'fail',
                'msg':'acceptable sampling rate is 16kHz',
                'wav_temp' :wav_temp, 
                'data' : data}

    elif data.load().shape[0] != 1:
        return {'status' : 'fail',
                'msg' : 'mono channel audio required',
                'wav_temp' : wav_temp,
                'data' : data }
    

    return {'status': 'pass',
            'msg' : 'acceptable audio',
            'wav_temp' : wav_temp,
            'data' : data }

In [ ]:
def verifySpeaker(base64string):
    [status,msg,_,_].values() = decode_base64(base64string)
    
    if status = 'fail':
        return {'status': status,'msg' : msg}
    

    
    
    
    